In [113]:
# Import the BigQuery client library
from google.cloud import bigquery

# Set up BigQuery client
client = bigquery.Client()
state="SD"
# Your SQL query
query = """
SELECT *
FROM `roofai-404514.ML_DATA.SD_DATA`
"""

# Run the query and store results in Pandas DataFrame named df
df = client.query(query).to_dataframe()


In [114]:
# df.head()

In [115]:
# df.info()

In [116]:
# df.isnull().sum()

In [117]:
# # Create a new column 'success' based on conditions using apply
# df['success'] = df.apply(lambda row: 1 if (row['SOLD_PAST_12_MONTHS'] == 1) or (row['LISTED_PAST_12_MONTHS'] == 1) else 0, axis=1)


In [118]:
# df

In [119]:
df['success'].round().value_counts()

success
0    308082
1      6200
Name: count, dtype: Int64

In [120]:
import pandas as pd

# Assuming df is your Pandas DataFrame

# Define the desired number of rows for success values 1 and 0
num_rows_success_1 = 6200
num_rows_success_0 = 6200

# Extract rows with 'success' value of 1
df_success_1 = df[df['success'] == 1].sample(n=num_rows_success_1,replace=True)

# Extract rows with 'success' value of 0
df_success_0 = df[df['success'] == 0].sample(n=num_rows_success_0,replace=True)

# Extract remaining rows for test_df
test_df = df.drop(df_success_0.index)
test_df.drop(test_df[test_df['success'] == 1].index, inplace=True)

# Create new_df by concatenating rows for success values 1 and 0
train_df = pd.concat([df_success_1, df_success_0])

# Display the sizes of the resulting DataFrames
print(f"Original DataFrame size: {len(df)}")
print(f"train_df size: {len(train_df)}, test_df size: {len(test_df)}")


Original DataFrame size: 314282
train_df size: 12400, test_df size: 301949


In [121]:
train_df['success'].round().value_counts()

success
1    6200
0    6200
Name: count, dtype: Int64

In [122]:
test_df['success'].round().value_counts()

success
0    301949
Name: count, dtype: Int64

In [123]:

# Drop multiple columns
columns_to_drop = ['ATTOM_ID','POTENTIAL_PROFIT','DATE_SOLD','SOL_PAST_12_MONTHS','DAYS_SINCE_LISTED','LAST_LISTED_DATE', 'SOLD_PAST_12_MONTHS','LISTED_PAST_12_MONTHS']
train_df  = train_df .drop(columns=columns_to_drop, axis=1)


In [124]:
train_df

,PROPERTY_ADDRESS_ZIP,COMPANY,STATE,COUNTY,MSA_CODE,DAYS_AGE_PROPERTY,LAND_USE_CODE,OWNER_OCCUPIED,TAX_ASSESSED_VALUE_TOTAL,AVM,...,RATIO_OWNERSHIP_TO_PROP_AGE,POOL,ALTV,FIRST_LOAN_AMORTIZED,EST_AVAILABLE_EQUITY,LOANTYPE_EQUITY,LOANTYPE_REFI,TOTAL_LOANS,TOTAL_OWNERS,success
104587,57006,1,46,011,15100,43435,1001,0,162900,287900,...,NaN,<NA>,<NA>,<NA>,<NA>,0,0,3,1,1
234867,57701,0,46,103,39660,25915,1001,1,230300,290400,...,NaN,<NA>,<NA>,<NA>,<NA>,0,0,0,1,1
152828,57108,0,46,083,43620,<NA>,1001,1,276304,484900,...,NaN,<NA>,27,130816,357184,1,0,3,1,1
230239,57702,1,46,103,39660,21170,1001,0,303000,667100,...,NaN,<NA>,<NA>,<NA>,<NA>,0,0,0,1,1
199170,57103,1,46,099,43620,48545,1001,0,104400,207700,...,NaN,<NA>,51,<NA>,80423,1,0,3,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77832,,0,46,121,,<NA>,7777,0,<NA>,<NA>,...,NaN,<NA>,<NA>,<NA>,<NA>,0,0,0,4,0
8593,,0,46,069,,<NA>,<NA>,0,<NA>,<NA>,...,NaN,<NA>,<NA>,<NA>,<NA>,0,0,0,4,0
144675,57783,0,46,081,43940,6570,1001,1,423810,446000,...,NaN,<NA>,<NA>,<NA>,<NA>,0,0,3,2,0
26768,57701,0,46,103,39660,27375,1101,0,362600,216000,...,NaN,<NA>,<NA>,<NA>,<NA>,0,0,3,1,0


In [125]:
# train_df.to_gbq('')  

client = bigquery.Client()
table_id = F'roofai-404514.ML_DATA.{state}_training_data'

job = client.load_table_from_dataframe(
    train_df, table_id
)


In [126]:
# train_df.to_gbq('')  

client = bigquery.Client()
table_id = f'roofai-404514.ML_DATA.{state}_testing_data'

job = client.load_table_from_dataframe(
    test_df, table_id
)
